# Getting Tensor Comprehensions

```shell
$ conda install -y -c pytorch -c tensorcomp tensor_comprehensions
```
Note: Won;t work on your mac, this is my Ubuntu server.

In [1]:
import tensor_comprehensions as tc
import torch

In [2]:
lang = """
def matmul(float(M,N) A, float(N,K) B) -> (output) {
  output(i, j) +=! A(i, kk) * B(kk, j)
}
"""

In [3]:
matmul = tc.define(lang, name="matmul")
mat1, mat2 = torch.randn(3, 4).cuda(), torch.randn(4, 5).cuda()
out = matmul(mat1, mat2)

[WARNING]: No mapping options passed, 'naive' type mapping options will be used and will likely have bad performance. See help(your_layer.__call__) for setting mapping options.


In [4]:
out

Variable containing:
 4.4190  6.8120  0.6187  1.4199  2.9564
-0.1778  3.2029  0.6903  1.1478  3.6830
-0.1244 -0.2070  2.8454 -0.1517  2.3613
[torch.cuda.FloatTensor of size 3x5 (GPU 0)]

## PyTorch layers in Tensor Comprehensions 

### Use of mapping option

Default Mapping: We provide various default options that can be chosen to closely represent the kernel. The defaults provided are:

- `pointwise, color=red`: if kernel resembles a pointwise operation
- `mlp`: if kernel resembles an Linear layer operation
- `conv`: if kernel resembles a convolution operation
- `group_conv`: if kernel resembles a group convolution operation
- `naive`: if none of the above, then chose naive default <-- This is why we get the warning
<font color='red'>bar</font>

In [5]:
# Specifying mapping options
matmul = tc.define(lang, name="matmul")
mat1, mat2 = torch.randn(100, 400).cuda(), torch.randn(400, 500).cuda()
out2 = matmul(mat1, mat2, options=tc.Options("mlp"))

In [6]:
out2

Variable containing:
-3.0480e+01 -1.0359e+01  2.3496e+01  ...  -2.3371e+00  2.3665e+01  5.3475e+00
 1.5563e+01  1.9449e+01 -3.7515e+01  ...   4.1372e+01  2.6417e+01 -2.4326e+01
-4.2722e+00  4.8939e+00 -2.9344e+01  ...  -2.0365e+01 -1.1882e+01  1.9065e+01
                ...                   ⋱                   ...                
-1.1980e+01 -3.4460e+01  1.1720e+01  ...  -1.0116e+01  3.3799e+00  1.1610e+01
-1.0537e+01 -1.8616e+01 -2.7911e+01  ...  -1.1084e+00 -6.0975e+00  8.4499e-01
 1.1400e+00 -2.0821e+01  3.2536e+01  ...  -4.1840e+00  2.0562e+01  2.9990e+01
[torch.cuda.FloatTensor of size 100x500 (GPU 0)]

In [7]:
# Using reduction operators
# providing different input sizes for the same comprehension

matmul = tc.define(lang, name="matmul")
mat1, mat2 = torch.randn(3, 4).cuda(), torch.randn(4, 5).cuda()
out = matmul(mat1, mat2)

# different input sizes
mat3, mat4 = torch.randn(100, 400).cuda(), torch.randn(400, 500).cuda()
out2 = matmul(mat3, mat4)
print(out)
print(out2)

[WARNING]: No mapping options passed, 'naive' type mapping options will be used and will likely have bad performance. See help(your_layer.__call__) for setting mapping options.
[WARNING]: No mapping options passed, 'naive' type mapping options will be used and will likely have bad performance. See help(your_layer.__call__) for setting mapping options.
Variable containing:
 0.8828  2.3928  3.0727  1.5787  0.4792
-1.6771 -1.3249  1.4003 -0.7534 -0.1971
 1.8660  3.4502  3.7494  2.9027  0.3870
[torch.cuda.FloatTensor of size 3x5 (GPU 0)]

Variable containing:
-3.0021e+01  3.1965e-01  4.9152e+00  ...  -5.6621e-01  1.0574e+01  1.6474e+01
-4.2599e+00 -1.4569e+01 -1.3952e+01  ...  -9.8632e+00 -3.9812e+01  4.8284e+01
 1.7316e+01  2.1750e+01  6.8386e+00  ...  -6.8485e+00  2.0356e+01 -6.8485e+00
                ...                   ⋱                   ...                
 2.1712e+01  1.7247e+01  5.6285e+00  ...   1.1536e+01  2.5222e+01 -1.5900e+00
-7.0703e+00  2.4916e+00 -1.9119e+01  ...  -2.379

#### Multiple TC definitions

Let’s say you want to define all of your TCs in one string and later use that string for running different operations defined in the string. You an do so easily. You can define a <font color='blue'>lang</font> variable that holds the TC definition for all your operations. Every time you want to run a different operation, you can make a <font color='blue'>tc.define</font> call on the <font color='blue'>lang</font> variable, specify the <font color='blue'>name</font> corresponding to the operation definition and get the TC layer for it. Below is an example for how to do this:

In [8]:
lang = """
def matmul(float(M,N) A, float(N,K) B) -> (output) {
  output(i, j) +=! A(i, kk) * B(kk, j)
}
def abs(float(M, N) A) -> (O1) {
  O1(m, n) = fabs(A(m, n))
}
"""
matmul = tc.define(lang, name="matmul")
mat1, mat2 = torch.randn(3, 4).cuda(), torch.randn(4, 5).cuda()
out = matmul(mat1, mat2)

abs = tc.define(lang, name="abs")
A = torch.randn(3, 4).cuda()
out = abs(A)

[WARNING]: No mapping options passed, 'naive' type mapping options will be used and will likely have bad performance. See help(your_layer.__call__) for setting mapping options.
[WARNING]: No mapping options passed, 'naive' type mapping options will be used and will likely have bad performance. See help(your_layer.__call__) for setting mapping options.


In [9]:
out

Variable containing:
 0.7115  0.0360  0.5991  0.7690
 2.6534  0.5275  0.6045  1.1807
 0.2736  0.2717  0.8221  1.3991
[torch.cuda.FloatTensor of size 3x4 (GPU 0)]

#### Writing layers with Scalars

- **Option 1**: Pass a constants dictionary to the tc.define call as demo'ed below

In [10]:
lang = """
def avgpool(float(B, C, H, W) input) -> (output) {{
    output(b, c, h, w) += input(b, c, h * {sH} + kh, w * {sW} + kw) where kh in 0:{kH}, kw in 0:{kW}
}}
"""
avgpool = tc.define(lang, name="avgpool", constants={"sH":1, "sW":1, "kH":2, "kW":2})
inp = torch.ones(32, 3, 10, 10).cuda()
out4 = avgpool(inp, options=tc.Options("mlp"))

In [11]:
out4

Variable containing:
(0 ,0 ,.,.) = 
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
     ...       ⋱       ...    
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4

(0 ,1 ,.,.) = 
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
     ...       ⋱       ...    
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4

(0 ,2 ,.,.) = 
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
     ...       ⋱       ...    
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
     ⋮ 

(1 ,0 ,.,.) = 
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
     ...       ⋱       ...    
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4
   4   4   4  ...    4   4   4

(1 ,1 ,.,.) = 
   4   4   4  ...    4  

#### Option 2 : Format the string using python regex 

In [12]:
import re
LANG="""
def avgpool(float(B, C, H, W) input) -> (output) {
    output(b, c, h, w) += input(b, c, h * <sh> + kh, w * <sw> + kw) where kh in 0:<kH>, kw in 0:<kW>
}
"""
sH, sW, kH, kW = 1, 1, 2, 2
LANG = re.sub('<sh>', str(sH), LANG)
LANG = re.sub('<sw>', str(sW), LANG)
LANG = re.sub('<kH>', str(kH), LANG)
LANG = re.sub('<kW>', str(kW), LANG)
avgpool = tc.define(LANG, name="avgpool")
inp = torch.ones(1, 1, 4, 4).cuda()
out5 = avgpool(inp)

[WARNING]: No mapping options passed, 'naive' type mapping options will be used and will likely have bad performance. See help(your_layer.__call__) for setting mapping options.


### Manually injecting external CUDA code

If you have an external efficient CUDA code that you want to use rather than the CUDA code that TC generates, you can inject your code easily. For this, you need to create a string which has the CUDA code you want to inject and you need to pass the name of the kernel and the CUDA code string to the `tc.define` call.

As an example:

In [13]:
lang = """
def add(float(N) A, float(N) B) -> (output) {
    output(i) = A(i) + B(i) + 1
}
"""

cuda_code = """
extern "C"{
__global__ void my_add(float* __restrict__ output, const float* __restrict__ A, const float* __restrict B)
{
    int t = threadIdx.x;
    output[t] = A[t] + B[t];
}
}
"""

add = tc.define(lang, name="add", inject_kernel="my_add", cuda_code=cuda_code)
a, b = torch.randn(100).cuda(), torch.randn(100).cuda()
out6 = add(a, b, grid=[1, 1, 1], block=[100, 1, 1])

[WARNING]: No mapping options passed, 'naive' type mapping options will be used and will likely have bad performance. See help(your_layer.__call__) for setting mapping options.


In [14]:
out6

Variable containing:
 1.5874
 0.4552
-0.2078
 1.1713
-1.0618
 0.4382
-1.2206
-1.6584
-0.5253
 0.2965
 0.6096
 3.1664
-0.7341
-0.7384
-0.2571
-2.1293
 0.2517
 3.5692
-1.0520
-1.1691
 0.9402
-0.1988
-0.3305
-2.9940
-0.0040
 0.5639
-0.2765
 1.6709
 2.6564
 0.6843
-0.8549
-0.8264
 0.2881
-0.6200
-1.5192
-0.2855
-0.8486
 0.0760
 1.2471
 1.3496
-2.9373
 0.1909
 0.7367
 2.9864
-2.4552
-0.2387
 1.4945
-0.4196
 0.5897
 0.1584
-0.5091
 0.1469
 0.4208
 1.4985
 0.5420
-0.7054
-1.7710
-0.1103
-0.1707
-2.4924
 1.9609
 0.3161
-1.4042
-1.2554
 1.2221
 0.0631
 1.0591
-0.6237
 0.4839
-0.0996
 0.6492
-0.1002
-1.2970
-0.3378
 1.1395
 0.5586
 0.3014
-2.0492
 0.6551
 0.7252
-0.6835
 1.1143
-2.4575
 1.4247
 0.1678
-0.1944
-1.4052
 3.2134
 0.9723
 2.3606
-0.9835
-2.0611
 4.5014
-1.0328
 1.1690
 1.5700
 0.4216
-1.7446
-1.5761
-0.2709
[torch.cuda.FloatTensor of size 100 (GPU 0)]

### Built-in Functions

TC allows using some CUDA built-in functions as well when defining the TC language. During the execution, CUDA API will be called for those built-in functions. For example, let’s say we want to use `fmax` CUDA function in our TC language.

In [15]:
LANG = """
def relu(float(B,M) I) -> (O1){
  O1(b, m) = fmax(I(b, m), 0)
}
"""
relu = tc.define(LANG, name="relu")
inp = torch.randn(100, 128).cuda()
out7 = relu(inp, options=tc.Options("mlp"))

In [16]:
out7

Variable containing:
 0.0000  0.2150  1.5466  ...   0.0000  0.0000  0.0000
 0.7299  1.5156  1.0762  ...   0.0000  1.2979  0.7923
 0.4733  0.0000  0.0459  ...   0.0000  0.0302  0.0000
          ...             ⋱             ...          
 0.0000  1.4061  0.0000  ...   0.0000  0.0855  0.0000
 0.0000  0.0000  0.0000  ...   0.0000  0.9500  0.0000
 0.0000  0.1261  0.7098  ...   0.0000  0.1167  0.0000
[torch.cuda.FloatTensor of size 100x128 (GPU 0)]

### For more CUDA Math, go here 

[CUDA Math Doc](http://docs.nvidia.com/cuda/cuda-math-api/group__CUDA__MATH__SINGLE.html#group__CUDA__MATH__SINGLE)

## Machine Learning Layers Database

There is a database of about 30 machine learning layers that are used across various types of neural networks. 

If you want to use one of the layers in the database, you can query this database in your code easily. The database can be accessed by calling tc.database. This database is a dictionary of TC name to the TC definition. Each entry in the dictionary looks like: `{tc_name: {"lang": language, "grad": grad_language}}` where `tc_name` is the name of the operation, `lang` is the tc language describing that operation, `grad` is the TC language describing the gradient of that operation. The `grad` is optional entry.

In [ ]:
# for instance with tgamma
LANG = """
def relu(float(B,M) I) -> (O1){
  O1(b, m) = fmax(I(b, m), 0)
}
"""
relu = tc.define(LANG, name="tgamma")
inp = torch.randn(100, 128).cuda()
out8 = relu(inp, options=tc.Options("mlp"))

We already have computed the matmul for mat1 and mat2 

### Pooling Layers 

#### Average Pooling

In [20]:
LANG = """
        def avgpool(float(B, C, H, W) input) -> (output) {{
            output(b, c, h, w) +=! input(b, c, h * {sH} + kh, w * {sW} + kw) / ({kH} * {kW}) where kh in 0:{kH}, kw in 0:{kW}
        }}
        """